## 1. Importing Libraries

We will first import the custom Libraries and Function along with standard Python libraries. <br>
The custom Class - pcm_class, pcm_block and function - copy_to_path should be in the same folder where we want to run the code.

In [ ]:
import os 
import subprocess
# Change the current working directory
os.chdir(r"C:\Users\Desktop\BEM\pcm_paper")

from pcm_class import IDF_EDITING
from pcm_block import MaterialProperty
from copy_to_path import copy_to_dir

## 2. Copy file from one directorty and added extension to name and copied to other directory

In order to edit the files and keep the original files intact, we will copy the file and add extension (i.e. _pcm23 for PCM with melting temperature of 23). The files will be saved in a new directory. The dir1 is the folder in local computer (we used windows OS here) where the IDF files are located. The files will be saved at dir2. Based on user convenient he/she can choose the dir2 and extension.

In [ ]:
import os
# # Paths

dir1 = r"C:\Users\Desktop\BEM\ASHRAE901_OfficeSmall__STD2013"
dir2 = r"C:\Users\Desktop\BEM\ASHRAE_SO_2013\pcm23"
copy_to_dir(dir1, dir2,file_type='idf', extension='_pcm23.')

## 3. Finding idf file names in the directory

The following code will find the idf files from a given directyory (or folder). <br>
Here, file_path is the folder where the IDF files are saved after copying from dir1 and added extension to each file name.

In [ ]:
## List of idf in a folder
import os
def idf_file_list(file_path):
    # Import necessary libraries
    import os, glob, shutil
    # Change directory to old idf files to work with
    os.chdir(file_path)

    # Getting names of all idf files only
    files = glob.glob("*.idf")
    return files

file_path = r"C:\Users\Desktop\BEM\ASHRAE_SO_2013\pcm23"
idf_files = idf_file_list(file_path)
print(idf_files)

In order to insert PCM properties to building envelope, we need to know the names of Wall, Roof, or Floor inside IDF files. The following code will find these names and show results in lists. In this case, all the IDF files uses similar naming for Wall, Roof and Floor. Therefore, we just extracted the name from first IDF file (i.e. idf_files[0])

In [ ]:
## Find the name of wall/roof/floor
idf_file = idf_files[0] # choose just any one of the idf file

def find_envelope_name(file_path):
    lines = []
    with open(file_path, 'r') as rf:
        lines = rf.readlines()

    lists = lines.copy()

    # Create a dictionary with line numbers and contents
    words = {}
    for idx, item in enumerate(lists):
        words[idx] = item.strip()

    # Prepare lists to store wall, roof, and floor names
    wall_names = []
    roof_names = []
    floor_names = []

    # Iterate through the lines to find BuildingSurface:Detailed objects
    for key, val in words.items():
        if 'BuildingSurface:Detailed' in val:
            current_key = key
            # Extract the surface name and exclude anything after '!'
            surface_name = words[current_key + 3].split(',')[0].strip()

            if "Wall" in words[current_key + 2]:
                wall_names.append(surface_name)
            elif "Roof" in words[current_key + 2]:
                roof_names.append(surface_name)
            elif "Floor" in words[current_key + 2]:
                floor_names.append(surface_name)

    return wall_names, roof_names, floor_names

# Example usage
file_path = rf"C:\Users\Desktop\BEM\ASHRAE_SO_2013\pcm23\{idf_file}"
wall_names, roof_names, floor_names = find_envelope_name(file_path)

## set ensure one name appeared only once
print("Walls:", set(wall_names))
print("Roofs:", set(roof_names))
print("Floors:", set(floor_names))

## 4. User input for material proprties 

We will first select in which of the envelope we will insert PCM material layes. In this study, we only choose walls. Each envelope may have seperate layers. In some case, the _split_word may be only 'Layer ' or the name inside IDF. Open up IDF file and look for how many layers are there. In this case, we added PCM into Layer 1. 
The original wall looks something like this.
<pre>
  Construction,
    nonres_ext_wall_grd,                     !- Name
	F07 25mm stucco,                         !- Outside Layer
	G01 16mm gypsum board,                   !- Layer #2
	Nonres_Exterior_Wall_Insulation_grd,     !- Layer #3
	G01 16mm gypsum board;                   !- Layer #4
</pre>

After running the code the PCM layer will be added like the following:
<pre>
Construction,
    nonres_ext_wall_grd,                     !- Name
    F07 25mm stucco,                         !- Outside Layer
    PCMBoard,                                !- Layer 1
    G01 16mm gypsum board,                   !- Layer #2
    Nonres_Exterior_Wall_Insulation_grd,     !- Layer #3
    G01 16mm gypsum board;                   !- Layer #4
</pre>

### In order to run the simulations without adding any PCM, user can directly jump to step  6. 

In [ ]:
# List of envelope names (walls, roofs, and floors) you want to edit
walls = ['nonres_ext_wall_grd']
roofs = ['nonres_roof']
floors = ['Core_bot_ZN_5_Floor_Ffactor']

# Choose the envelope names you want to edit (can be a single item or a list of surfaces)
_envelope_names = walls  # You can use 'walls', 'roofs', 'floors', or a combination like ['nonres_roof', 'nonres_ext_wall_top']

# Check the IDF file for envelope layer. (e.g. 'Layer ' or 'Layer #' based on naming Layer 1 or Layer #1)
_split_word = 'Layer #' # Where PCM layer will be added
_pcm_layer = "!- Layer 1"

# Example PCM to use
pcm_mat = MaterialProperty('pcmboard')

# PCM23
pcm_mat.material(0.025, 0.27, 1000, 1132)
pcm_mat.phasechange(-30.0, 0, 21, 60000, 23, 72000, 100, 160000) # Base

# PCM18
# pcm_mat.material(0.025, 0.2, 1000, 2000)
# pcm_mat.phasechange(-20.0, 0, 16, 75000, 19, 275000, 50, 330000) # # PCM18
# pcm_mat.phasechange(-20.0, 0, 19, 75000, 22, 275000, 50, 330000) # # PCM21
# pcm_mat.phasechange(-20.0, 0, 22, 75000, 25, 275000, 50, 330000) # # PCM24

# PCM28
# pcm_mat.material(0.025, 0.25, 814, 2150)
# pcm_mat.phasechange(-20.0, 0, 28, 103200, 28.1, 226300, 100, 481000)
_pcm_block = pcm_mat.pcmblock()

# 5. Editing idf files with PCM properties

In [ ]:
# Loop through each IDF file
for file_name in idf_files:
    file01 = os.path.abspath(file_name)  # Get absolute path of IDF file
    file_path = file01

    count = 0 # counting number of enveoples to which PCM is added
    for _envelope_name in _envelope_names:
        # Create an instance of IDF_EDITING for each surface
        file_object = IDF_EDITING(file01, _envelope_name, _split_word, _pcm_layer, _pcm_block)
        
        # Editing the entire IDF file with PCM properties
        if count<=0:
            file_object.create_idf()
            
        else:
        # It will only create idf without appending PCM properties
        # For PCM addition to more than one envelope, pcm properties need to include once in IDF.
            file_object.create_idf__no_appending()

        count+=1

    print(f"Finished processing {file_name}")

# ## Testing with one file only so loop is commented out  
# file01 = os.path.abspath(idf_files[1])
# file_object = IDF_EDITING(file01, _envelope_name, _split_word, _pcm_layer,  _pcm_block)
# file_object.create_idf()

# 6. Creating new directories for each idf file and run simulation 

In [ ]:
import os
import subprocess

# List of simulation results' directory
output_directory_list = []

def find_matching_epw(epw_directory, idf_filename):
    base_name = idf_filename.replace("_pcm23", "").split('_')[-1].split('.')[0]
    # base_name = idf_filename.replace("_pcm23", "").split('_')[-1].split('.')[0]
    
    for epw_file in os.listdir(epw_directory):
        if epw_file.endswith('.epw') and base_name.lower() in epw_file.lower():
            return os.path.join(epw_directory, epw_file)
    return None

def run_energyplus(idf_directory, epw_directory, eplus_dir):
    for file_name in os.listdir(idf_directory):
        if file_name.endswith('.idf'):
            idf_filepath = os.path.join(idf_directory, file_name)
            base_name = os.path.splitext(file_name)[0]
            output_directory = os.path.join(idf_directory, f"results_{base_name}")

            if not os.path.exists(output_directory):
                os.makedirs(output_directory)

            output_directory_list.append(output_directory) 
            
            matched_epw = find_matching_epw(epw_directory, file_name)
            if matched_epw:
                cl_st = [
                    f"{eplus_dir}\\EnergyPlus",
                    "--readvars",  # included to create a .csv file of the results
                    f"--output-directory={output_directory}",
                    f"--weather={matched_epw}",
                    idf_filepath,
                ]

                result = subprocess.run(cl_st, capture_output=True, text=True)

                if result.returncode == 0:
                    print(f"Simulation for {file_name} completed successfully.")
                else:
                    print(f"Simulation for {file_name} failed.")
                    print("Error:", result.stderr)
            else:
                print(f"No matching .epw file found for {file_name}.")


idf_directory = r'C:\Users\Desktop\BEM\ASHRAE_SO_2013\pcm23'  # Path to .idf files
epw_directory = r'C:\Users\Desktop\BEM\ASHRAE901_epw'  # Path to .epw files
eplus_dir = r'C:\EnergyPlusV22-1-0'  # Path to EnergyPlus installation

run_energyplus(idf_directory, epw_directory, eplus_dir)

# 7. Getting results

In [ ]:
## Printing the new created folders with simulation results 
print(output_directory_list)
results_dir = output_directory_list

In [ ]:
import os
import glob
import pandas as pd
from bs4 import BeautifulSoup

# ---------- Helpers ----------
def htm_file_list(directory):
    """Returns a list of .htm files in the given directory."""
    if not os.path.isdir(directory):
        raise NotADirectoryError(f"{directory} is not a valid directory.")
    return glob.glob(os.path.join(directory, "*.htm"))

def extract_heating_cooling_fan_loads(html_file):
    """Extract total heating, cooling, and fan load from EnergyPlus HTML report."""
    with open(html_file, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f, 'html.parser')

    end_uses_tag = soup.find('b', string="End Uses")
    if not end_uses_tag:
        return None, None, None

    table_tag = end_uses_tag.find_next('table')
    if not table_tag:
        return None, None, None

    heating = cooling = fan = 0.0
    rows = table_tag.find_all('tr')[1:]  # skip header
    for row in rows:
        cols = row.find_all('td')
        if not cols:
            continue
        end_use = cols[0].get_text(strip=True).lower()
        values = [float(col.get_text(strip=True)) for col in cols[1:] if col.get_text(strip=True)]
        if end_use == "heating":
            heating = sum(values)
        elif end_use == "cooling":
            cooling = sum(values)
        elif end_use == "fans":
            fan = sum(values)

    return heating, cooling, fan

def extract_zone_sensible_load_manual(soup, section_name):
    """Extract Zone Sensible Heating & Cooling totals from HTML."""
    section_tag = soup.find("b", string=section_name)
    if not section_tag:
        return None

    table_tag = section_tag.find_next("table")
    if not table_tag:
        return None

    rows = table_tag.find_all("tr")
    if not rows:
        return None

    header = [td.get_text(strip=True) for td in rows[0].find_all("td")]

    try:
        col_idx = next(i for i, h in enumerate(header) if "Calculated Design Load" in h)
    except StopIteration:
        return None

    total = 0.0
    for row in rows[1:]:
        cells = row.find_all("td")
        if len(cells) <= col_idx:
            continue
        value_str = cells[col_idx].get_text(strip=True)
        try:
            value = float(value_str)
            total += value
        except ValueError:
            continue

    return total

# ---------- Main Processing ----------
df_results = pd.DataFrame(columns=[
    'Folder Name', 'File Name',
    'Heating Load (GJ)', 'Cooling Load (GJ)', 'Fan Load (GJ)', 'Total HVAC Load (GJ)',
    'Peak Heating Demand (W)', 'Peak Cooling Demand(W)'
])

for folder in output_directory_list:
    folder = os.path.abspath(folder)
    try:
        htm_files = htm_file_list(folder)
        folder_name = os.path.basename(folder)

        for htm_file in htm_files:
            with open(htm_file, 'r', encoding='utf-8') as f:
                soup = BeautifulSoup(f, 'html.parser')

            heating, cooling, fan = extract_heating_cooling_fan_loads(htm_file)
            zone_heating = extract_zone_sensible_load_manual(soup, "Zone Sensible Heating")
            zone_cooling = extract_zone_sensible_load_manual(soup, "Zone Sensible Cooling")

            total_hvac = (heating or 0) + (cooling or 0) + (fan or 0)

            new_row = pd.DataFrame([[folder_name, os.path.basename(htm_file),
                                     heating, cooling, fan, total_hvac,
                                     zone_heating, zone_cooling]],
                                   columns=df_results.columns)
            if not new_row.isna().all().all():
                df_results = pd.concat([df_results, new_row], ignore_index=True)

    except NotADirectoryError as e:
        print(e)

# Save results
output_csv = os.path.join(os.getcwd(), "heating_cooling_zone_loads.csv")
df_results.to_csv(output_csv, index=False)
print(df_results)


In [ ]:
df_results

# F8 Metal deleted from Medium Office